In [ ]:
# Install libraries (jalankan jika belum terinstal)
!pip install tensorflow tensorflow-recommenders ipywidgets pandas scikit-learn

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow.keras.layers import Input, Dense, Dot
from tensorflow.keras.models import Model
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split
from ipywidgets import interact, widgets

In [ ]:
# Load dataset
dataset = pd.read_csv('dataset_loker_downsynd.csv')

# Display dataset
print("Dataset Overview:")
print(dataset.head())

Dataset Overview:
      Nama Pekerjaan                Perusahaan      Lokasi          Gaji  \
0      Asisten Dapur         PT Roti Nusantara     Jakarta  Rp 2.500.000   
1      Petugas Kebun        Taman Kota Mandiri     Bandung  Rp 2.000.000   
2  Operator Komputer           PT Digital Jaya    Surabaya  Rp 3.000.000   
3  Pembersih Ruangan       PT Bersih Sejahtera  Yogyakarta  Rp 2.200.000   
4  Fotografer Pemula  Studio Kreatif Nusantara    Denpasar  Rp 3.500.000   

                                 Deskripsi Pekerjaan       Kategori Minat  \
0  Membantu koki menyiapkan bahan masakan dan men...              Memasak   
1  Merawat tanaman, menyiram bunga, dan menjaga k...             Berkebun   
2  Mengoperasikan komputer untuk tugas-tugas sede...      Teknologi dasar   
3  Membersihkan ruangan kerja dan menjaga kebersi...  Lingkungan dan alam   
4  Mengambil foto sederhana dan membantu pengedit...      Fotografi, Seni   

                                Kemampuan Dibutuhkan  \
0  Men

In [ ]:
# Kategori Minat
kategori_minat = [
    "Seni", "Musik", "Menari", "Kerajinan tangan", "Berkebun", "Memasak",
    "Merawat hewan", "Menggambar atau melukis", "Teknologi dasar (komputer, tablet)",
    "Fotografi", "Lingkungan dan alam"
]

# Kategori Kemampuan
kategori_kemampuan = [
    "Membaca dasar", "Menulis sederhana", "Mendengarkan instruksi", "Berbicara dengan orang lain",
    "Kerja tim", "Ketelitian", "Koordinasi tangan dan mata", "Kesabaran", "Kreativitas",
    "Mengikuti langkah-langkah sederhana", "Menggunakan alat sederhana", "Pemecahan masalah sederhana",
    "Mandiri dengan supervisi", "Menjaga kebersihan", "Mengatur waktu", "Ketahanan fisik ringan"
]

# Kategori Kondisi Kesehatan
kategori_kesehatan = [
    "Menggunakan kursi roda", "Membutuhkan kacamata", "Menggunakan alat bantu dengar",
    "Kesulitan berdiri terlalu lama", "Tidak boleh mengangkat beban berat",
    "Membutuhkan lingkungan kerja tenang", "Tidak cocok di suhu panas", "Tidak cocok di suhu dingin",
    "Alergi bahan kimia tertentu", "Membutuhkan istirahat berkala"
]


In [ ]:
# Isi nilai NaN dengan string kosong
dataset['Kategori Minat'] = dataset['Kategori Minat'].fillna('')
dataset['Kemampuan Dibutuhkan'] = dataset['Kemampuan Dibutuhkan'].fillna('')
dataset['Kondisi Kesehatan'] = dataset['Kondisi Kesehatan'].fillna('')

# Pisahkan nilai menjadi daftar
dataset['Kategori Minat'] = dataset['Kategori Minat'].str.split(', ')
dataset['Kemampuan Dibutuhkan'] = dataset['Kemampuan Dibutuhkan'].str.split(', ')
dataset['Kondisi Kesehatan'] = dataset['Kondisi Kesehatan'].str.split(', ')

# Filter hanya kategori valid
dataset['Kategori Minat'] = dataset['Kategori Minat'].apply(
    lambda x: [i for i in x if i in kategori_minat]
)
dataset['Kemampuan Dibutuhkan'] = dataset['Kemampuan Dibutuhkan'].apply(
    lambda x: [i for i in x if i in kategori_kemampuan]
)
dataset['Kondisi Kesehatan'] = dataset['Kondisi Kesehatan'].apply(
    lambda x: [i for i in x if i in kategori_kesehatan]
)

# Display dataset setelah preprocessing
print("Dataset after preprocessing:")
print(dataset.head())


Dataset after preprocessing:
      Nama Pekerjaan                Perusahaan      Lokasi          Gaji  \
0      Asisten Dapur         PT Roti Nusantara     Jakarta  Rp 2.500.000   
1      Petugas Kebun        Taman Kota Mandiri     Bandung  Rp 2.000.000   
2  Operator Komputer           PT Digital Jaya    Surabaya  Rp 3.000.000   
3  Pembersih Ruangan       PT Bersih Sejahtera  Yogyakarta  Rp 2.200.000   
4  Fotografer Pemula  Studio Kreatif Nusantara    Denpasar  Rp 3.500.000   

                                 Deskripsi Pekerjaan         Kategori Minat  \
0  Membantu koki menyiapkan bahan masakan dan men...              [Memasak]   
1  Merawat tanaman, menyiram bunga, dan menjaga k...             [Berkebun]   
2  Mengoperasikan komputer untuk tugas-tugas sede...                     []   
3  Membersihkan ruangan kerja dan menjaga kebersi...  [Lingkungan dan alam]   
4  Mengambil foto sederhana dan membantu pengedit...      [Fotografi, Seni]   

                                Kemampu

In [ ]:
# Encode fitur menggunakan MultiLabelBinarizer
minat_encoder = MultiLabelBinarizer()
skills_encoder = MultiLabelBinarizer()
kondisi_encoder = MultiLabelBinarizer()

minat_encoded = minat_encoder.fit_transform(dataset['Kategori Minat'])
skills_encoded = skills_encoder.fit_transform(dataset['Kemampuan Dibutuhkan'])
kondisi_encoded = kondisi_encoder.fit_transform(dataset['Kondisi Kesehatan'])

# Gabungkan fitur dengan bobot
job_features = np.hstack([
    minat_encoded * 0.5,  # Bobot 50% untuk Minat
    skills_encoded * 0.25,  # Bobot 25% untuk Kemampuan
    kondisi_encoded * 0.25  # Bobot 25% untuk Kondisi
])

# Normalisasi
scaler = StandardScaler()
job_features = scaler.fit_transform(job_features)

# Salin job_features untuk user_features
user_features = job_features.copy()


In [ ]:
# Bagi data menjadi train-test
user_train, user_test, job_train, job_test = train_test_split(user_features, job_features, test_size=0.2, random_state=1)


In [ ]:
# Input untuk user dan job
user_input = Input(shape=(job_features.shape[1],), name="user_input")
job_input = Input(shape=(job_features.shape[1],), name="job_input")

# User Network
user_net = Dense(128, activation='relu')(user_input)
user_net = Dense(64, activation='relu')(user_net)
user_net = Dense(32, activation='relu')(user_net)

# Job Network
job_net = Dense(128, activation='relu')(job_input)
job_net = Dense(64, activation='relu')(job_net)
job_net = Dense(32, activation='relu')(job_net)

# Kesamaan menggunakan Dot Product
similarity = Dot(axes=1)([user_net, job_net])

# Bangun model
model = Model(inputs=[user_input, job_input], outputs=similarity)
model.compile(optimizer='adam', loss='mse', metrics=["accuracy"])
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_input (InputLayer)   │ (None, 29)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ job_input (InputLayer)    │ (None, 29)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 128)            │          3,840 │ user_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_15 (Dense)          │ (None, 128)            │          3,840 │ job_input[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 64)             │          8,256 │ dense_12[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_16 (Dense)          │ (None, 64)             │          8,256 │ dense_15[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_14 (Dense)          │ (None, 32)             │          2,080 │ dense_13[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_17 (Dense)          │ (None, 32)             │          2,080 │ dense_16[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dot_2 (Dot)               │ (None, 1)              │              0 │ dense_14[0][0],        │
│                           │                        │                │ dense_17[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 28,352 (110.75 KB)

 Trainable params: 28,352 (110.75 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Buat target dummy untuk pelatihan
dummy_target = np.ones((user_train.shape[0], 1))

# Latih model
model.fit([user_train, job_train], dummy_target, epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.0461 - loss: 0.8074 - val_accuracy: 0.1111 - val_loss: 0.6379
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.2791 - loss: 0.4862 - val_accuracy: 0.7222 - val_loss: 0.2753
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7093 - loss: 0.3488 - val_accuracy: 0.7778 - val_loss: 0.1256
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9044 - loss: 0.0972 - val_accuracy: 1.0000 - val_loss: 0.0402
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.9092 - loss: 0.0755 - val_accuracy: 1.0000 - val_loss: 0.0447
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.0508 - val_accuracy: 1.0000 - val_loss: 0.0466
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 1.0000 - loss: 0.0269 - val_accuracy: 1.0000 - val_loss: 0.0307
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9965 - loss: 0.0155 - val_accuracy: 1.0000 - val_loss: 0.0245


In [ ]:
# Step 8: Latih Model dengan Target Similarity
def create_similarity_targets(job_features, user_features):
    targets = []
    for job, user in zip(job_features, user_features):
        similarity = np.sum(job * user)  # Dot product untuk kesamaan
        targets.append(similarity)
    return np.array(targets)

similarity_targets = create_similarity_targets(job_train, user_train)
similarity_targets = (similarity_targets - similarity_targets.min()) / (similarity_targets.max() - similarity_targets.min())
similarity_targets = np.expand_dims(similarity_targets, axis=1)

model.fit([user_train, job_train], similarity_targets, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0113 - loss: 0.0038 - val_accuracy: 0.0000e+00 - val_loss: 0.0066
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0113 - loss: 0.0031 - val_accuracy: 0.0000e+00 - val_loss: 0.0069
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0182 - loss: 0.0031 - val_accuracy: 0.0000e+00 - val_loss: 0.0068
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0165 - loss: 0.0032 - val_accuracy: 0.0000e+00 - val_loss: 0.0066
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0156 - loss: 0.0022 - val_accuracy: 0.0000e+00 - val_loss: 0.0068
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0057 - loss: 0.0020 - val_accuracy: 0.0000e+00 - val_loss: 0.0067
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0100 - loss: 0.0021 - val_accuracy: 0.0000e+00 - val_loss: 0.0066
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0070 - loss: 0.0014 - val_accuracy: 0.

In [ ]:
# Step 9: Fungsi Rekomendasi
def encode_user_preferences(preferences):
    user_minat = minat_encoder.transform([[preferences[0]]])
    user_skills = skills_encoder.transform([[preferences[1]]])
    user_kondisi = kondisi_encoder.transform([[preferences[2]]])

    user_vector = np.hstack([
        user_minat * 0.5,
        user_skills * 0.25,
        user_kondisi * 0.25
    ])
    return scaler.transform(user_vector)

In [ ]:
def recommend_jobs(new_user_preferences, job_features, model, dataset, top_n=5):
    new_user_vector = encode_user_preferences(new_user_preferences)
    new_user_repeated = np.repeat(new_user_vector, job_features.shape[0], axis=0)
    predictions = model.predict([new_user_repeated, job_features])
    sorted_indices = np.argsort(-predictions.flatten())

    print("Top Recommendations:")
    for i in sorted_indices[:top_n]:
        recommended_job = dataset.iloc[i]
        print(f"Nama Pekerjaan: {recommended_job['Nama Pekerjaan']}")
        print(f"Perusahaan: {recommended_job['Perusahaan']}")
        print(f"Lokasi: {recommended_job['Lokasi']}")
        print(f"Kategori Minat: {recommended_job['Kategori Minat']}")
        print(f"Kemampuan Dibutuhkan: {recommended_job['Kemampuan Dibutuhkan']}")
        print(f"Kondisi Kesehatan: {recommended_job['Kondisi Kesehatan']}")
        print("-" * 40)

In [ ]:
# Preferensi pengguna baru
new_user_preferences = ['Seni', 'Kreativitas', 'Membutuhkan lingkungan kerja tenang']

# Panggil fungsi rekomendasi
recommend_jobs(new_user_preferences, job_test, model, dataset)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Top Recommendations:
Nama Pekerjaan: Asisten Penjahit
Perusahaan: Konveksi Nusantara
Lokasi: Solo
Kategori Minat: ['Seni', 'Kerajinan tangan']
Kemampuan Dibutuhkan: ['Ketelitian', 'Kreativitas', 'Koordinasi tangan dan mata']
Kondisi Kesehatan: ['Membutuhkan lingkungan kerja tenang']
----------------------------------------
Nama Pekerjaan: Perancang Kerajinan
Perusahaan: Sanggar Kerajinan Daerah
Lokasi: Solo
Kategori Minat: ['Kerajinan tangan', 'Seni']
Kemampuan Dibutuhkan: ['Ketelitian', 'Koordinasi tangan dan mata', 'Kesabaran']
Kondisi Kesehatan: ['Tidak cocok di suhu dingin']
----------------------------------------
Nama Pekerjaan: Pekerja Roti Tradisional
Perusahaan: Roti Klasik Nusantara
Lokasi: Yogyakarta
Kategori Minat: ['Memasak']
Kemampuan Dibutuhkan: ['Ketelitian', 'Mengikuti langkah-langkah sederhana', 'Kreativitas']
Kondisi Kesehatan: []
----------------------------------------
Nama Pekerjaan: Petugas Museum
Perusahaan: Museum Nasional 

In [ ]:
# Simpan model ke dalam format HDF5
model.save('rekomendasi_pekerjaan.h5')

print("Model saved to 'rekomendasi_pekerjaan.h5'")

Model saved to 'job_recommender_model.h5'
